# Nettoyer le jeu ***suite***

~~~~
Capteur:
* InletTemp-sensor
* SetTemp-sensor
* Drive-sensor
* Mode-sensor
~~~~

### 1. Nettoyer le jeu
* [x] Exploration des colonnes et drop des non nécessaires
* [x] Noter les intervalles ou listes pour chaque capteur
* [x] Features engineering (transformation des dates, merges, etc...)
* [x] Exclure les outliers n'étant pas dans les intervalles ou listes
* [x] Vérifier la présence de doublons
* [x] Exploration des données
* [ ] API météo?

### 2. Modèle ML
* Modèle de Régression:
  * RandomTreeRegressor
  * Gradient Boosting (linéaire, tree)
  * KernelRidge (linéaire, rbf, etc...)

In [2]:
import sqlalchemy
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

pd.set_option("display.max_columns", None)

from sqlalchemy  import create_engine

db_type = "mysql"
driver = "mysqlconnector"
login = "root"
password = "MYSQLR00tP455w0rd!!!"
host = "localhost"
port = 3306
db_name = "client_main"

engine = create_engine(f"{db_type}://{login}:{password}@{host}:{port}/{db_name}")

measures_query = "SELECT * FROM DataMeasurements"
sensors_query = "SELECT * FROM Sensors"

cleaned_df = pd.read_csv("cleaned_datas.csv")
measures_df = pd.read_sql(measures_query, engine)
sensors_df = pd.read_sql(sensors_query, engine)

In [3]:
cleaned_df

,DataMeasurementId,DateTime,SensorId,Value,Raw,Name,EquipmentId
0,1,2021-10-18 10:13:28.415,1,19.1,19.1,InletTemp-sensor,1
1,2,2021-10-18 10:13:28.446,2,29.0,29.0,SetTemp-sensor,1
2,4,2021-10-18 10:13:28.481,5,2.0,HEAT,Mode-sensor,1
3,5,2021-10-18 10:13:28.492,7,22.5,22.5,InletTemp-sensor,2
4,6,2021-10-18 10:13:28.516,8,16.0,16.0,SetTemp-sensor,2
...,...,...,...,...,...,...,...
63732,68086,2021-11-21 15:49:49.258,13,21.9,21.9,InletTemp-sensor,3
63733,68087,2021-11-21 15:51:49.347,31,23.1,23.1,InletTemp-sensor,6
63734,68088,2021-11-21 15:51:49.386,49,26.0,26.0,InletTemp-sensor,9
63735,68089,2021-11-21 15:56:49.565,7,22.1,22.1,InletTemp-sensor,2


In [18]:
from datetime import datetime, timedelta

test_d = datetime.strptime(cleaned_df.loc[0, "DateTime"][:-7], "%Y-%m-%d %H:%M")
print(test_d)
test_d - timedelta(hours= 1)

2021-10-18 10:13:00


datetime.datetime(2021, 10, 18, 9, 13)

In [ ]:
for idx, row in cleaned_df.iterrows():
    print(type(row["DateTime"]))
    print(row["DateTime"])

    if idx >= 0:
        break

In [4]:
datas = measures_df.merge(sensors_df[["SensorId", "Name", "EquipmentId"]], on= "SensorId")
datas.sort_values(by= "DataMeasurementId", inplace= True)
datas.reset_index(drop= True, inplace= True)

datas

,DataMeasurementId,DateTime,SensorId,Value,Raw,Name,EquipmentId
0,1,2021-10-18 10:13:28.415,1,19.1,19.1,InletTemp-sensor,1
1,2,2021-10-18 10:13:28.446,2,29.0,29.0,SetTemp-sensor,1
2,3,2021-10-18 10:13:28.469,4,4.0,3005,FanSpeed-sensor,1
3,4,2021-10-18 10:13:28.481,5,2.0,HEAT,Mode-sensor,1
4,5,2021-10-18 10:13:28.492,7,22.5,22.5,InletTemp-sensor,2
...,...,...,...,...,...,...,...
68085,68086,2021-11-21 15:49:49.258,13,21.9,21.9,InletTemp-sensor,3
68086,68087,2021-11-21 15:51:49.347,31,23.1,23.1,InletTemp-sensor,6
68087,68088,2021-11-21 15:51:49.386,49,26.0,26.0,InletTemp-sensor,9
68088,68089,2021-11-21 15:56:49.565,7,22.1,22.1,InletTemp-sensor,2


In [5]:
sensors_df.drop(["Technical_Name", "DataTypeId", "UnitId", "Active", "FatherId", "IdLocal_Hub", 
    "SubType", "TransmissionMode", "DirectUrl", "BoxId", "IdLocal_Setup"], axis= 1, inplace= True)

# names = [x for x in sensors_df["Name"].unique() if x not in ["FanSpeed-sensor", "ErrCode-sensor"]]

# sensors_df = sensors_df[sensors_df["Name"].isin(names)].reset_index(drop= True)

equipt_1 = sensors_df[sensors_df["EquipmentId"] == 1]

interv_listes = {}

for capt in equipt_1["Name"].unique():
    interv_listes[capt] = []
    idx = equipt_1.loc[equipt_1["Name"] == capt].index.values[0]
    for col in equipt_1.columns[-3:]:
        value = equipt_1.loc[idx, col]
        if type(value) == np.float64:
            if not np.isnan(value):
              interv_listes[capt].append(value)
        elif value is not None and type(value) == str:
            tmp_vals = value.split(";")
            for tmp_val in tmp_vals:
                interv_listes[capt].append(tmp_val.split("|")[0])

interv_listes

{'InletTemp-sensor': [16.0, 32.0],
 'SetTemp-sensor': [16.0, 32.0],
 'Drive-sensor': ['0', '2'],
 'FanSpeed-sensor': ['3003', '3005'],
 'Mode-sensor': ['COOL', 'HEAT', 'FAN'],
 'ErrCode-sensor': []}

Je veux trouver les valeurs numériques de "Value" correspondant à "Raw" pour les capteurs "Drive-sensor" et "Mode-sensor".

In [6]:
str_capteurs = [x for x in datas["Name"].unique() if x not in ["InletTemp-sensor", 
    "SetTemp-sensor"]]

vals_corresp = {}

for capt, vals in interv_listes.items():
    if capt in str_capteurs:
        vals_corresp[capt] = {}
        for val in vals:
            vals_corresp[capt][val] = datas.loc[(datas["Name"] == capt) & (datas["Raw"] 
                == val), "Value"].unique().tolist()

vals_corresp



{'Drive-sensor': {'0': [1.0], '2': [0.0]},
 'FanSpeed-sensor': {'3003': [1.0], '3005': [4.0]},
 'Mode-sensor': {'COOL': [], 'HEAT': [2.0], 'FAN': [4.0]}}

1. Drive-sensor:
   1. 2-> mode automatique: "Value" == 0
   2. 0-> mode manuel: "Value" == 1
2. FanSpeed-sensor:
   1. 3003: "Value" == 1
   2. 3005: "Value" == 4
3. Mode-sensor:
   1. Cool: pas de valeur dans le jeu
   2. Heat: "Value" == 2
   3. Fan: "Value" == 4

Donc il faut que il faut que je trouve les probabilités d'apparitions des valeurs de "Value":
* "Value" == 1: Drive-sensor, FanSpeed-sensor
* "Value" == 4: FanSpeed-sensor, Mode-sensor

In [7]:
datas = datas[datas["Value"].notna()]

truc_list = []
for idx, row in datas.iterrows():
    if row["Name"] in ["InletTemp-sensor", "SetTemp-sensor"]:
        truc_list.append(np.float64(row["Raw"]))
    else:
        truc_list.append(row["Raw"])

datas["Raw"] = truc_list.copy()

df = pd.DataFrame([], columns= datas.columns)

for key, val in interv_listes.items():
    tmp_df = datas[datas["Name"] == key]
    if key in ["InletTemp-sensor", "SetTemp-sensor"]:
        tmp_ddf = tmp_df[(tmp_df["Raw"] >= val[0]) & (tmp_df["Raw"] <= val[1])]
        df = pd.concat([df, tmp_ddf])
    else:
        tmp_ddf = tmp_df[tmp_df["Raw"].isin(val)]
        df = pd.concat([df, tmp_ddf])

df = df.sort_values(by= "DateTime").reset_index(drop= True)

C:\Users\nanar\AppData\Local\Temp/ipykernel_10352/172852748.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  datas["Raw"] = truc_list.copy()


In [8]:
f_datas = pd.DataFrame([], columns= datas.columns)

for i in range(1, 20):
    tmp_df = df[df["EquipmentId"] == i].copy()
    tmp_df["diff_time"] = pd.to_datetime(tmp_df['DateTime'].astype(str)).diff(1).dt.total_seconds()
    
    f_datas = pd.concat([f_datas, tmp_df])

f_datas.reset_index(drop= True, inplace= True)

In [9]:
f_datas

,DataMeasurementId,DateTime,SensorId,Value,Raw,Name,EquipmentId,diff_time
0,1,2021-10-18 10:13:28.415,1,19.1,19.1,InletTemp-sensor,1,NaN
1,2,2021-10-18 10:13:28.446,2,29.0,29.0,SetTemp-sensor,1,0.031
2,3,2021-10-18 10:13:28.469,4,4.0,3005,FanSpeed-sensor,1,0.023
3,4,2021-10-18 10:13:28.481,5,2.0,HEAT,Mode-sensor,1,0.012
4,260,2021-10-18 11:34:33.376,1,19.2,19.2,InletTemp-sensor,1,4864.895
...,...,...,...,...,...,...,...,...
67818,67958,2021-11-21 13:00:05.206,109,22.3,22.3,InletTemp-sensor,19,1859.899
67819,67959,2021-11-21 13:00:05.217,110,19.0,19.0,SetTemp-sensor,19,0.011
67820,67960,2021-11-21 13:00:05.228,111,0.0,2,Drive-sensor,19,0.011
67821,67962,2021-11-21 13:00:05.249,113,2.0,HEAT,Mode-sensor,19,0.021


In [10]:
f_datas .to_csv("cleaned_datas_2.csv", index= False)

Je vais regarder le capteur qui précède celui qui m'intéresse et:
* Analyser les fréquences
* Analyser le temps

In [11]:
results_dict = {}
for capt in f_datas["Name"].unique() : # str_capteurs:
    results_dict[capt] = {}

# Je parcours les équipements un par un
for i in range(1, 20):
    tmp_df = f_datas[f_datas["EquipmentId"] == i].reset_index(drop= True)

    # print(tmp_df.loc[len(tmp_df) -1, "Name"])

    # Pour chaque individu de l'équipement à partir d'un premier "SetTemp-sensor"
    start = tmp_df[tmp_df["Name"] == "SetTemp-sensor"].iloc[0].name
    set_val = tmp_df.loc[start, "Value"]
    for j in range(start, len(tmp_df)):
        
        # Je récupère le capteur actuel et le précédent
        actual_capt = tmp_df.loc[j, "Name"]
        prev_capt = tmp_df.loc[j -1, "Name"]

        # Si la clé/valeur n'existe pas dans le dictionnaire, je la crée
        if results_dict[prev_capt].get(actual_capt) is None \
            and actual_capt in ["InletTemp-sensor", "SetTemp-sensor"]:
            results_dict[prev_capt][actual_capt] = 0
        if results_dict[prev_capt].get(actual_capt) is None \
            and actual_capt in ["Drive-sensor", "FanSpeed-sensor", "Mode-sensor"]:
            results_dict[prev_capt][actual_capt] = {}
            for v in vals_corresp[actual_capt].values():
                if len(v) > 0:
                    results_dict[prev_capt][actual_capt][v[0]] = 0

        # Je récupère la valeur de "Value"
        value = tmp_df.loc[j, "Value"]
        
        # Si "Value" est compris entre 16 et 32 inclus
        if 16 <= value <= 32:
            if value == int(value):
                # Si "Value" est une valeur entière, je regarde la différence 
                # de temps avec le relevé précédent
                # Si le temps est inférieur à 2s, il s'agit de "SetTemp-sensor"
                if tmp_df.loc[j, "diff_time"] <= 2:
                    results_dict[prev_capt]["SetTemp-sensor"] += 1
                else:
                    # Sinon je fais la différence entre la valeur précédente de 
                    # "SetTemp_sensor" et "Value"
                    difference = set_val - value
                    # Si la différence est un nombre décimal, il s'agit de "InletTemp-sensor"
                    if difference == int(difference):
                        results_dict[prev_capt][actual_capt] += 1
                    else:
                        results_dict[prev_capt]["InletTemp-sensor"] += 1
            else:
                # Si "Value" est une valeur décimale, il s'agit de "InletTemp-sensor"
                results_dict[prev_capt]["InletTemp-sensor"] += 1

        elif value == 0:
            results_dict[prev_capt]["Drive-sensor"][0] += 1
        elif value == 1:
            results_dict[prev_capt][actual_capt][1] += 1
        elif value == 2:
            results_dict[prev_capt]["Mode-sensor"][2] += 1
        elif value == 4:
            results_dict[prev_capt][actual_capt][4] += 1

        else:
            # pass
            print(f"Valeur abérrante: {value}")


        if actual_capt == "SetTemp-sensor":
            set_val = tmp_df.loc[j, "Value"]



sommes_dict ={}

for keys, values in results_dict.items():
    if sommes_dict.get(keys) is None:
        sommes_dict[keys] = {}
    for vs in values.values():
        if type(vs) == dict:
            for k, v in vs.items():
                if k == 0:
                    if sommes_dict[keys].get(k) is None:
                        sommes_dict[keys][k] = 0
                    sommes_dict[keys][k] += v
                elif k == 1:
                    if sommes_dict[keys].get(k) is None:
                        sommes_dict[keys][k] = 0
                    sommes_dict[keys][k] += v
                elif k == 2:
                    if sommes_dict[keys].get(k) is None:
                        sommes_dict[keys][k] = 0
                    sommes_dict[keys][k] += v
                elif k == 4:
                    if sommes_dict[keys].get(k) is None:
                        sommes_dict[keys][k] = 0
                    sommes_dict[keys][k] += v


for keys, values in results_dict.items():
    somme_float = np.sum([v for k, v in values.items() if k in ["InletTemp-sensor", "SetTemp-sensor"]])
    for ks, vs in values.items():
        if ks in ["InletTemp-sensor", "SetTemp-sensor"]:
            results_dict[keys][ks] = np.round(vs / somme_float *100, 3)
        elif ks in ["Drive-sensor", "FanSpeed-sensor", "Mode-sensor"]:
            for k, v in vs.items():
                results_dict[keys][ks][k] = np.round(v / sommes_dict[keys][k] *100, 3)



import pprint

pprint.pprint(results_dict)


{'Drive-sensor': {'Drive-sensor': {0.0: 100.0, 1.0: 3.025},
                  'FanSpeed-sensor': {1.0: 96.975, 4.0: 100.0},
                  'InletTemp-sensor': 61.435,
                  'Mode-sensor': {2.0: 100.0, 4.0: 0.0},
                  'SetTemp-sensor': 38.565},
 'FanSpeed-sensor': {'Mode-sensor': {2.0: 100.0, 4.0: 100.0}},
 'InletTemp-sensor': {'Drive-sensor': {0.0: 100.0, 1.0: 100.0},
                      'InletTemp-sensor': 89.72,
                      'SetTemp-sensor': 10.28},
 'Mode-sensor': {'Drive-sensor': {0.0: 100.0, 1.0: 100.0},
                 'InletTemp-sensor': 96.408,
                 'SetTemp-sensor': 3.592},
 'SetTemp-sensor': {'Drive-sensor': {0.0: 100.0, 1.0: 100.0},
                    'FanSpeed-sensor': {1.0: 0.0, 4.0: 100.0},
                    'InletTemp-sensor': 72.811,
                    'Mode-sensor': {2.0: 100.0, 4.0: 0.0},
                    'SetTemp-sensor': 27.189}}


In [12]:
valeur = 30.1

print(f"Selon l'historique des données, les probabilités sont les suivantes:")
# Parcours du jeu de données équipement par équipement
for i in range(1, 20):
    tmp_df = f_datas[f_datas["EquipmentId"] == i].reset_index(drop= True)

    print(f"\nÉquipement {i}:")
    # Récupération du capteur du dernier relevé
    prev_capt = tmp_df.loc[len(tmp_df) -1, "Name"]

    # Extraction du premier rang de probabilité
    tmp_proba = results_dict[prev_capt]
    
    print_bool = False
    # Si "valeur" est dans l'invervalle [16, 32]
    if 16 <= valeur <= 32:
        for k, v in tmp_proba.items():
            if type(v) == np.float64:
                print(f"{v}% que ce soit le capteur {k}")
                print_bool = True
    else:
        for key, value in tmp_proba.items():
            if type(value) == dict:
                for k, v in value.items():
                    if valeur == k:
                        print(f"{v}% que ce soit le capteur {key}")
                        print_bool = True

    if not print_bool:
        print(f"Cas inconnu d'après l'historique des données.")


    # print(f"{i} - {tmp_proba}")

Selon l'historique des données, les probabilités sont les suivantes:

Équipement 1:
10.28% que ce soit le capteur SetTemp-sensor
89.72% que ce soit le capteur InletTemp-sensor

Équipement 2:
10.28% que ce soit le capteur SetTemp-sensor
89.72% que ce soit le capteur InletTemp-sensor

Équipement 3:
10.28% que ce soit le capteur SetTemp-sensor
89.72% que ce soit le capteur InletTemp-sensor

Équipement 4:
10.28% que ce soit le capteur SetTemp-sensor
89.72% que ce soit le capteur InletTemp-sensor

Équipement 5:
96.408% que ce soit le capteur InletTemp-sensor
3.592% que ce soit le capteur SetTemp-sensor

Équipement 6:
10.28% que ce soit le capteur SetTemp-sensor
89.72% que ce soit le capteur InletTemp-sensor

Équipement 7:
10.28% que ce soit le capteur SetTemp-sensor
89.72% que ce soit le capteur InletTemp-sensor

Équipement 8:
10.28% que ce soit le capteur SetTemp-sensor
89.72% que ce soit le capteur InletTemp-sensor

Équipement 9:
10.28% que ce soit le capteur SetTemp-sensor
89.72% que ce 

Le code ci-dessus me semble le plus adapté.

* Maintenant, je lance ce code puis je demande le numéro de l'équipement et la valeur.
* Je reprends la dernière valeur entrée dans l'historique de l'équipement
* Je retourne la probabilité en fonction de la dernière entrée
* Puis, je mets à jour le code ci-dessus et demande une nouvelle valeur ('q' pour quitter)

## 2. Modèle ML

Je vais commencer par mettre en forme le jeu

créer un jeu train et un test.